## Setup

In [31]:
import numpy as np
import os
import PIL
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pathlib

In [32]:
print(tf.__version__)

2.8.0


To train a model with this dataset you will want the data:

* To be well shuffled.
* To be batched.
* Batches to be available as soon as possible.

These features can be added using the `tf.data` API. For more details, visit the [Input Pipeline Performance](../../guide/performance/datasets.ipynb) guide.

In [ ]:
# AUTOTUNE IS EEN EXPIRIMENT VOOR LATER, ZORGT ER NU VOOR DAT MODEL NIET GOED LAADT VOOR TRAINING

# AUTOTUNE = tf.data.AUTOTUNE 

# # train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
# # val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# def configure_for_performance(ds):
#   ds = ds.cache()
#   ds = ds.shuffle(buffer_size=1000)
#   ds = ds.batch(batch_size)
#   ds = ds.prefetch(buffer_size=AUTOTUNE)
#   return ds

# train_ds = configure_for_performance(train_ds)
# val_ds = configure_for_performance(val_ds)
# test_ds = configure_for_performance(test_ds)

In [ ]:
# num_classes = 5 geen idee waarom model.summary niet werkt

# def create_model():
#   model = tf.keras.Sequential([
#     tf.keras.layers.Rescaling(1./255),
#     tf.keras.layers.Conv2D(32, 3, activation='relu'),
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.Conv2D(32, 3, activation='relu'),
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.Conv2D(32, 3, activation='relu'),
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(num_classes)
# ])

#   model.compile(optimizer='adam',
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

#   return model

# Create a basic model instance
# model = create_model()

# # Display the model's architecture
# model.summary()


In [14]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255, input_shape(img_height, img_width, 3)),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [15]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

Choose the `tf.keras.optimizers.Adam` optimizer and `tf.keras.losses.SparseCategoricalCrossentropy` loss function. To view training and validation accuracy for each training epoch, pass the `metrics` argument to `Model.compile`.

Note: You will only train for a few epochs so this tutorial runs quickly. 

In [18]:
# checkpoint_path = "training_1/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# # Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)
#                                                  # save_freq=5*batch_size) terug aan als batch omhoog gaat

model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10)
#   callbacks=[cp_callback])  # Pass callback to training


Epoch 1/10
306/306 [==============================] - ETA: 0s - loss: 1.2686 - sparse_categorical_accuracy: 0.4706
Epoch 1: saving model to training_1/cp.ckpt
306/306 [==============================] - 27s 89ms/step - loss: 1.2686 - sparse_categorical_accuracy: 0.4706 - val_loss: 1.0808 - val_sparse_categorical_accuracy: 0.5921
Epoch 2/10
306/306 [==============================] - ETA: 0s - loss: 1.0463 - sparse_categorical_accuracy: 0.6275
Epoch 2: saving model to training_1/cp.ckpt
306/306 [==============================] - 27s 89ms/step - loss: 1.0463 - sparse_categorical_accuracy: 0.6275 - val_loss: 0.8460 - val_sparse_categorical_accuracy: 0.6842
Epoch 3/10
306/306 [==============================] - ETA: 0s - loss: 0.7184 - sparse_categorical_accuracy: 0.7320
Epoch 3: saving model to training_1/cp.ckpt
306/306 [==============================] - 28s 90ms/step - loss: 0.7184 - sparse_categorical_accuracy: 0.7320 - val_loss: 0.8909 - val_sparse_categorical_accuracy: 0.7237
Epoch 4/10

In [ ]:
# os.listdir(checkpoint_dir)

In [49]:
# print (test_ds.get_single_elem)

InvalidArgumentError: Dataset had more than one element. [Op:DatasetToSingleElement]

In [98]:
def get_label(file_path):
    return file_path.split("\\")[-2]

In [100]:
#simplest predictions

pred = model.predict(test_ds)

pred = np.argmax(pred, axis=-1)

print(pred)

# original labels

actual = np.argmax(test_ds.map(get_label))

print(actual)


# conf_matrix = tf.math.confusion_matrix(labels=class_names, predictions=pred)

# print(conf_matrix)

[2 3 2 3 2 1 2 2 1 0 2 0 3 2 0 1 3 2 2 2 2 3 0 2 0 2 3 2 0 0 3 0 0 0 0 0 1
 0 0 0 2 3 3 0 3 0 2 3 2 1 1 0 3 1 0 2 0 0 2 0 2 3 3 2 3 3 0 0 2 0 0 2 2 2
 2 0 2 0 1 0 2 3 3 2 0 1 1 2 3 2 2 2 3 3 0 0 2 0 0 1 0 1 3 3 0 1 0 3 0 0 2
 2 2 0 2 2 0 3 2 3]


TypeError: in user code:


    TypeError: tf__get_label() takes 1 positional argument but 2 were given


In [ ]:
# #predict tf?

# predict_ds = tf.data.Dataset.from_tensor_slices(images).batch(32)
# result = model.predict(predict_ds, steps = 10)
# print(result.shape)

In [25]:
# predictions met plot

#hier kies je welke index je iets mee wilt
i = 1

#dit is het plaatje van index i
img = train_ds[i]

print (img)

# img_array = tf.keras.utils.img_to_array(img)
# img_array = tf.expand_dims(img_array, 0) # Change dims to fit model

# predictions = model.predict(img_array)
# score = tf.nn.softmax(predictions[0])

# print(
#     "This image most likely belongs to {} with a {:.2f} percent confidence."
#     .format(labelNames[np.argmax(score)], 100 * np.max(score))
# )

# # # print de label naam van index i
# print(labelNames[trainLabels[i]])

# # #laat plaatje zien
# plt.imshow(img)

# print(img.shape)
# # print(img)


TypeError: 'BatchDataset' object is not subscriptable

vragen:

labels zijn 0-1-2-3 ipv blotch, normal , rot en scab

prediction is np array ipv %

opslaan en ophalen met checkpoints?

hoe grijp ik elementen aan uit een tf dataset?